# Results for TSMCN-8-L-229 with SNR=inf. Noise added test data is analyzed after experimental classification

# SVM with various feature extraction methods

### conclusion, can only achieve 85-87% accuracy

In [ ]:
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import MaxNLocator
import numpy as np
from numpy import asarray
import pandas as pd
import math
import seaborn as sns  #heat map
import glob # batch processing of images

import datetime
import matplotlib.font_manager as fm
import random
import sys
import os

from sklearn.datasets import make_regression
from sklearn.metrics import confusion_matrix    #confusion matrix


# Collect all the font names available to matplotlib
font_names = [f.name for f in fm.fontManager.ttflist]
# print(font_names)

from scipy import signal
from scipy import interpolate

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve 
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score


from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF

#Sklearn model saving and loading
from joblib import dump, load

if '../../' not in sys.path:
    sys.path.append('../../')

from aimos.spectral_datasets.THz_datasets import THz_data

from aimos.misc.utils import simple_plotter


#Set random seed
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
# tf.random.set_seed(42)  
# tf.random.get_global_generator().reset_from_seed(42)
np.random.seed(42)
random.seed(42)

from oneida import THz_mixture_data
from oneida_utils import concentrations_to_one_hot_encode, create_mixture_names
from oneida_utils import simple_spectrum_fig, simple_plot_raw_scores, plot_spectrum_with_scores, multiclass_roc_auc_score, multiclass_sensitivity_specificity_score, multiclass_sensitivity_threshold_score
from oneida_scoring_tools import calc_AMCAS, is_cui_present, is_cui_present_in_mult
from aimos.misc.utils import classifier_internals
from aimos.misc.utils import clf_post_processor
from oneida_utils import mixture_names_to_one_hot_encode
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from oneida_grad_cam import grad_cam

from sklearn import svm
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from stats import stats
stats(n_compounds=8)

# Retrieve training(D)+ validation(V) mixtures and its variables

In [ ]:
# initialize
TAAT = 0.001 
ASAT=0.01
RSAT=0.01

m = THz_mixture_data(resolution=0.016, pressure='1 Torr', verbosity=False)
m.initiate_THz_mixture_data(TAAT = TAAT, 
                               ASAT=ASAT, 
                               RSAT=RSAT)

reduced_labels = m.labels
reduced_labels.remove('')
reduced_labels.remove(' ')
reduced_labels.remove('Diluent')
print('reduced_labels', reduced_labels)


# data_filename = "datasets/TSMCN-5-L-229_DV_04-09-2022_time_22-26-37.pkl"
data_filename = "datasets/TSMCN-8-L-229_DV__TAAT_0.001_ASAT_0.01_RSAT_0.01_20-10-2022_time_23-16-29_class_cnt_90.pkl"
DV = pd.read_pickle(data_filename)
y = DV['y'].to_numpy()
mixture_names = DV['mixture_names'].to_numpy()
y_concentrations = DV[['y_c0', 'y_c1', 'y_c2','y_c3', 'y_c4', 'y_c5', 'y_c6', 'y_c7']].to_numpy()
X = DV.drop(['y','mixture_names', 'y_c0', 'y_c1', 'y_c2','y_c3', 'y_c4', 'y_c5', 'y_c6', 'y_c7'],axis=1).to_numpy()

final_neuron_number = np.unique(y, axis=0).shape[0]
print('Number of neurons in the final layer :', final_neuron_number)

print('labels from class:', m.labels)

# preview one test mixture spectra using simple plotter

In [ ]:
# idx = 239
# simple_plotter(m.frequencies,X[idx],linewidth=0.5,color='black',label=mixture_names[idx], 
#                    majorsize=6,minorsize=2,width=1, labelsize=8,legendsize=3, legendloc=2,  
#                    labelpad=4,fontsize='medium',fontweight='bold',
#                   xmajormplloc=0.5,xminormplloc=0.2, tickdirection='out')

# print(y_concentrations[idx])
# print(reduced_labels)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(mixture_names)

mixture_types=le.classes_
print(mixture_types)

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



lda = LinearDiscriminantAnalysis(n_components=None)
X_transformed = lda.fit(X, y).transform(X)
print(X_transformed.shape)

In [ ]:
# from sklearn.datasets import load_digits
from sklearn.decomposition import KernelPCA
# X, _ = load_digits(return_X_y=True)


kPCA = KernelPCA(n_components=20, kernel='linear')
X_transformed = kPCA.fit_transform(X)
print(X_transformed.shape)

In [ ]:

X_csr = csr_matrix(X)
svd = TruncatedSVD(n_components=8, n_iter=7, random_state=42)
SVD = svd.fit(X_csr)

X_transformed = SVD.transform(X_csr)

print(svd.explained_variance_ratio_)

print(svd.explained_variance_ratio_.sum())

print(svd.singular_values_)

print(X_transformed.shape)

In [ ]:
#split intro train and validation set

#seeds used 123,237, 786
from sklearn.model_selection import train_test_split

global_indices=range(0, X.shape[0])
print(global_indices)

# (np.expand_dims(X,-1)
TRAIN_SIZE=0.60
VAL_SIZE=1-TRAIN_SIZE

x_train, x_val, y_train, y_val, train_indices, val_indices = train_test_split(X_transformed, y, global_indices, train_size=TRAIN_SIZE,
                                                   test_size=VAL_SIZE,
                                                   random_state=786,
                                                    stratify=y

                                                   )

print('X_train shape:', x_train.shape)
print('y_ohe_train shape:', y_train.shape)

print('X_val shape:', x_val.shape)
print('y_ohe_val shape:', y_val.shape)


print("All:", np.bincount(y) / float(len(y))*100  )
print("Training:", np.bincount(y_train) / float(len(y_train))*100  )
print("Validation:", np.bincount(y_val) / float(len(y_val))*100  )


In [ ]:
#OneVsRest (SVM-Linear Kernel)

#Measure time elapsed
t_start = datetime.datetime.now()

classifier_OVR = OneVsRestClassifier(SVC(kernel='rbf',C = 10000,decision_function_shape = 'ovo',random_state=1)).fit(x_train, y_train)


pred_y = classifier_OVR.predict(x_val)


FCA_OVR=np.sum(pred_y == y_val) / float(len(y_val))
print("Fraction Correct[Accuracy]:", FCA_OVR)


cm_OVR = confusion_matrix(y_val, pred_y)
print(classification_report(y_val, pred_y))

t_end = datetime.datetime.now()
delta = t_end - t_start
Time_OVR=delta.total_seconds() * 1000

print('Time elaspsed: ', Time_OVR) # milliseconds


In [ ]:
# tf.keras.utils.plot_model(model, to_file="RESULTS/TSMCN_8_L_229.png", show_shapes=True, rankdir='TB', dpi=150)

In [ ]:

exps = ['2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Mix 50% Dilute CM-ACN.xlsx',
'2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Pure Mix CM-ACN.xlsx',
'2 Comp-mix_ 30 % CH3Cl - 70% CH3CN/Mix 90% Dilute CM-ACN.xlsx',
'3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL/0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 1.xlsx',
'3 Comp-mix_ 90+% CH3OH + 5-% CH3CN + 5-% CH3CL/0.9 CH3OH + 0.05 CH3CN + 0.05 CH3Cl - 2.xlsx',
'4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx',
'4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/90% Dilute in N2 - 0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN'       ]

true_label=[81,81,81,82,82,19,19]
exp_path = '../../data/Mixture_exp_data/'
exp_filepath = '4 Comp-mix_ 67% CH3OH + 30% CH3CHO + 2% CH3Cl + 1% CH3CN/0.67 CH3OH + 0.3 CH3CHO + 0.02 CH3Cl + 0.01 CH3CN - v2.xlsx'


def classify_exp(exp_path,exp_filepath,mixture_types,true_label, excel=True):
    all_unique_labels= mixture_types
    if excel:
        df_exp1 = pd.read_excel(exp_path + exp_filepath)
    else:
        df_exp1 = pd.read_csv(exp_path + exp_filepath)


    freq_exp1 = df_exp1[df_exp1.columns[0]].to_numpy()
    abs_exp1 = df_exp1[df_exp1.columns[1]].to_numpy()
    
#     fft_filter(freq_exp1, abs_exp1, factor=75)
    
    dfy_resampled= signal.resample(abs_exp1, len(m.frequencies))
    dfx_resampled= signal.resample(freq_exp1, len(m.frequencies))
#     import pdb; pdb.set_trace()    
    pred_exp_label = classifier_OVR.predict(SVD.transform(np.reshape(dfy_resampled, (1, 229) )))


    print('Experiment name: ',exp_filepath.split('/')[0])
    print('predicted index ', pred_exp_label)
    print('predicted label', mixture_types[pred_exp_label])
    

    
    return pred_exp_label
idx = 0 
for experiment in exps:
    classify_exp(exp_path,experiment,mixture_types, true_label[idx],excel=True)
    idx+=1

In [ ]:
np.reshape()

In [ ]:
np.expand_dims(dfy_resampled, axis=-1)

In [ ]:
pred_exp_label = classifier_OVR.predict(SVD.transform(dfy_resampled))

In [ ]:
#Sklearn model saving and loading
from joblib import dump, load
import datetime
from datetime import date, datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_time_%H-%M-%S")
model.save('model/' + model_name + '_' + dt_string + '.hdf5')
np.save('model/' + model_name + '_' + dt_string + 'history' + '.npy',history.history)
np.save('model/' + model_name + '_' + dt_string + 'epoch' + '.npy',history.epoch)


In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(x_val)

In [ ]:
y_ohe = concentrations_to_one_hot_encode(y_concentrations).astype('int64')
y_train_ohe = y_ohe[train_indices]
y_val_ohe = y_ohe[val_indices]
y_train_ohe_tensor = tf.convert_to_tensor(y_train_ohe, np.int64)
y_val_ohe_tensor = tf.convert_to_tensor(y_val_ohe, np.int64)

In [ ]:
plt.figure(dpi=150)
plt.scatter(history.epoch,history.history['accuracy'], color = 'red', label = 'training')
plt.scatter(history.epoch,history.history['val_accuracy'], color = 'blue', label = 'validation')
plt.legend(loc=4)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.savefig(r'RESULTS/results_figures/' + model_name + '_accuracies.png', bbox_inches='tight')

plt.figure(dpi=150)
plt.scatter(history.epoch,history.history['SparseCatCrossentropy'], color = 'red', label = 'training')
plt.scatter(history.epoch,history.history['val_SparseCatCrossentropy'],color = 'blue', label = 'validation')
plt.legend(loc=1)
plt.xlabel('Epoch')
plt.ylabel('Sparse categorical crossentropy loss')
plt.savefig(r'RESULTS/results_figures/'+ model_name + '_sparse_cat_losses.png', bbox_inches='tight')